# <b>Image classification batch feature demo</b>

The AIServiceVisionClient offers the image classification feature in batch mode. This notebook aims to provide overall clarity about the feature to the user in terms of requirements, usage and the output of the batch i.e. asynchronous API.<br>
<ul>
    <li>The output response files are stored at the object storage specified in <code>data/output_object_image_batch.json</code>. </li>
<li>The detected classes for a random input image are displayed in the last section of the notebook.</li>
</ul>

### Steps to run the notebook:
<details>
    <summary>Notebook session setup</summary>
    <ol>
        <li><font size="2">Installing the OCI Vision SDK</font></li>
        <li><font size="2">Installing other dependencies</font></li>
        <li><font size="2">Setup sample input images</font></li>
        <li><font size="2">Setup helper .py files</font></li>
    </ol>
</details>

<details>
    <summary>Importing the required modules</summary>
</details>

<details>
    <summary>Setting the input variables</summary>
     <font size="2">The user can change the input variables, if necessary. They have been assigned default values.</font>
</details>

<details>
    <summary>Running the main pipeline</summary>
    <font size="2">Run all cells to get the output in the <code>output</code> directory. </font><br>
</details>

### Notebook session setup
<details>
    <summary>Instructions</summary>
    <ul>
        <li><font size="2">The user needs to setup only once.</font></li>
        <li><font size="2">Uncomment the commented cells and run once to setup.</font></li>
        <li><font size="2">Comment back the same cells to avoid running again.</font></li>
    </ul>
</details>

#### Installing the OCI Vision SDK

In [1]:
# !wget "https://objectstorage.us-ashburn-1.oraclecloud.com/n/axhheqi2ofpb/b/vision-demo-notebooks/o/vision_service_python_client-0.3.45-py2.py3-none-any.whl"
# !pip install vision_service_python_client-0.3.45-py2.py3-none-any.whl
# !rm vision_service_python_client-0.3.45-py2.py3-none-any.whl

#### Installing other dependencies

In [2]:
# !pip install matplotlib==3.3.4
# !pip install pandas==1.1.5

#### Setup sample input images

In [25]:
# !wget "https://objectstorage.us-ashburn-1.oraclecloud.com/n/axhheqi2ofpb/b/vision-demo-notebooks/o/input_objects_image_batch.json"
# !wget "https://objectstorage.us-ashburn-1.oraclecloud.com/n/axhheqi2ofpb/b/vision-demo-notebooks/o/output_object_image_batch.json"
# !mkdir data
# !mv input_objects_image_batch.json data
# !mv output_object_image_batch.json data

#### Setup helper .py files

In [24]:
# !wget "https://objectstorage.us-ashburn-1.oraclecloud.com/n/axhheqi2ofpb/b/vision-demo-notebooks/o/analyze_image_batch_utils.py"
# !mkdir helper
# !mv analyze_image_batch_utils.py helper

### Imports

In [15]:
import time
import json
import io
from random import randint
import oci
from PIL import Image

from vision_service_python_client.models import output_location
from vision_service_python_client.ai_service_vision_client import AIServiceVisionClient
from vision_service_python_client.models.create_image_job_details import CreateImageJobDetails
from vision_service_python_client.models.image_classification_feature import ImageClassificationFeature
from helper.analyze_image_batch_utils import load_input_object_locations, load_output_object_location, display_classes, clean_output
from IPython.display import JSON

### Set input variables
<details>
    <summary><font size="3">input_location_path</font></summary>
    <font size="2">The file <code>data/input_objects_image_batch.json</code> specifies where the input images are to be taken from. Default values have been set. If needed, the user needs to provide the following in this file:
        <ul>
            <li><code>compartment_id</code> : Compartment ID</li>
            <li><code>input_objects</code>: List with the object locations in the following format-</li>
            <ul>
                <li><code>namespace</code> : Namespace name</li>
                <li><code>bucket</code> : Bucket name</li>
                <li><code>objects</code> : List of object names</li>
            </ul>
        </ul>
    </font>
</details>

<details>
    <summary><font size="3">output_location_path</font></summary>
    <font size="2">The file <code>data/output_object_image_batch.json</code> specifies where the output files will be stored. Default values have been set. If needed, the user needs to provide the following in this file:
        <ul>
            <li><code>namespace</code> : Namespace name</li>
            <li><code>bucket</code> : Bucket name</li>
            <li><code>prefix</code> : Prefix name</li>
        </ul>
    </font>
</details>

<details>
<summary><font size="3">max_results</font></summary>
    <font size="2">Provide the maximum number of results needed for image classification. This is an upper limit over the output classes, the API may detect lesser classes according to the image.</font><br>
</details>

In [16]:
input_location_path = 'data/input_objects_image_batch.json'
output_location_path = 'data/output_object_image_batch.json'
max_results = 5

### Authorize user config

In [17]:
config = oci.config.from_file('~/.oci/config')

### Load input and output object locations

In [18]:
compartment_id, input_location = load_input_object_locations(input_location_path)
output_location = load_output_object_location(output_location_path)

### Create AI service vision client and image job

In [19]:
ai_service_vision_client = AIServiceVisionClient(config=config)
create_image_job_details = CreateImageJobDetails()

image_classification_feature = ImageClassificationFeature()
image_classification_feature.max_results = max_results
features = [image_classification_feature]
create_image_job_details.features = features
create_image_job_details.compartment_id = compartment_id
create_image_job_details.input_location = input_location
create_image_job_details.output_location = output_location

res = ai_service_vision_client.create_image_job(create_image_job_details=create_image_job_details)

### Job submitted
The job is created and is in <code>ACCEPTED</code> state.

In [20]:
res_json = json.loads(repr(res.data))
clean_res = clean_output(res_json)
JSON(clean_res)

<IPython.core.display.JSON object>

### Job in progress
The job progress is tracked till completion with an interval of 5 seconds and is in <code>IN_PROGRESS</code> state.

In [21]:
job_id = res.data.id
print("Job ID :", job_id, '\n')
seconds = 0
res = ai_service_vision_client.get_image_job(image_job_id=job_id)

while res.data.lifecycle_state in ["IN_PROGRESS", "ACCEPTED"]:
    print("Job is IN_PROGRESS for " + str(seconds) + " seconds")
    time.sleep(5)
    seconds += 5
    res = ai_service_vision_client.get_image_job(image_job_id=job_id)

Job ID : ocid1.aivisionimagejob.oc1.iad.amaaaaaa74akfsaaf4qwvdww4fmhrrok3gefwnkgrzmg3ugxqc6e57rz62va 

Job is IN_PROGRESS for 0 seconds
Job is IN_PROGRESS for 5 seconds
Job is IN_PROGRESS for 10 seconds
Job is IN_PROGRESS for 15 seconds
Job is IN_PROGRESS for 20 seconds
Job is IN_PROGRESS for 25 seconds
Job is IN_PROGRESS for 30 seconds
Job is IN_PROGRESS for 35 seconds


### Job completed
The job is completed and is in <code>SUCCEEDED</code> state.

In [22]:
res_json = json.loads(repr(res.data))
clean_res = clean_output(res_json)
JSON(clean_res)

<IPython.core.display.JSON object>

### Display detected classes
The detected classes will be displayed in decreasing order of confidence level for a randomly selected image from the batch input.

In [23]:
object_storage_client = oci.object_storage.ObjectStorageClient(config)

index = randint(0, len(input_location.object_locations) - 1)
object_location = input_location.object_locations[index]

output_object_name = output_location.prefix + "/" + res.data.id + "/" + \
        output_location.namespace_name  + "_" + output_location.bucket_name + "_" + \
            object_location.object_name

res_json = object_storage_client.get_object(output_location.namespace_name, \
    output_location.bucket_name, object_name = output_object_name+".json").data.content
res_dict = json.loads(res_json)

print("Image :", object_location.object_name)
if res_dict['labels'] is not None:
    display_classes(res_dict['labels'])
else:
    print("No image classes detected.")

ServiceError: {'opc-request-id': 'iad-1:atx5LAPi8asrQNwF9FNo6ffKrYltuKA4hjfjNF8mWHfcrJ4euP9V63yOsfgfcMYL', 'code': 'BucketNotFound', 'message': "Either the bucket named 'async-demo' does not exist in the namespace 'axhheqi2ofpb' or you are not authorized to access it", 'status': 404}